In [1]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np

gender_folder = r"C:\Users\agboo\Downloads\Gender_Classification"
gender_class = ["women","men"]

# we will use this to traverse our features and append them along with their classes
# we will make every picture 100 X 120  this is due to most pictures being selfies / and or full body
row = 120
col = 100
feature = []
def training():
    for gender in range(2):
        path = os.path.join(gender_folder, gender_class[gender])
        # creating flag we will classify with either 0 or 1
        flag = gender_class.index(gender_class[gender]) 
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_COLOR)
                new_img = cv2.resize(img_array,(row,col))
                feature.append([new_img,flag])
            except:
                print(" Error: Corrupted File :( ")
training()
print(len(feature))

# import random so we can shuffle data
import random
random.shuffle(feature)

X = []
Y = []

for feat , ident in feature:
    X.append(feat)
    Y.append(ident)
X = np.array(X).reshape(-1,row,col,3) # features
Y = np.array(Y) # flags


## now we will save the data as pickle files to be loaded into the model
import pickle 
X_out = open("Feat.pickle","wb") # open file for data to then be dumped
pickle.dump(X, X_out)
X_out.close()

Y_out = open("Ident.pickle","wb") # the same is done for the flag list
pickle.dump(Y, Y_out)
Y_out.close()

#### the data will be loaded using rb (read binary when we start to construct the model




    
    

    

    

    
    
    

 Error: Corrupted File :( 
 Error: Corrupted File :( 
 Error: Corrupted File :( 
 Error: Corrupted File :( 
1581


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , Flatten , Dense , Activation , MaxPooling2D

from tensorflow.keras.callbacks import TensorBoard

import time
import pickle

X = pickle.load(open("Feat.pickle","rb"))
Y = pickle.load(open("Ident.pickle","rb"))

X = X/255.0 # this will normalize the feature array


model = Sequential() #  so this will create the model

def pretty(node):
    while node:
        print(str(node.val) + "->", end ='')
        node = node.next
    if node:
        print(node.val)
    else:
        print(None)
        
        

        
class ConvNode(): # class for reshape node
    def __init__(self, val = 0 , next = None):
        self.val = val
        self.next = next
    def conv_node():
        model.add(Conv2D(32,(3,3) , input_shape = X.shape[1:]))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size = (2,2)))
    conv_node() # call the function so that we can  execute the layer

class HiddenNode(): # class for hidden layers most likely will be the longest
    def __init__(self,val = 0 , next = None):
        self.val = val
        self.next = next
    def hidden(): # so hidden layers will have four layers
        model.add(Conv2D(32,(3,3)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size = (2,2)))
        
        model.add(Conv2D(32,(3,3)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size = (2,2))) 
        
        model.add(Conv2D(32,(3,3)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size = (2,2)))
        
        model.add(Conv2D(32,(3,3)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size = (2,2)))
    hidden() 

class FlattenNode(): # we need only one flatten node
    def __init__(self,val = 0 ,next = None):
        self.val = val
        self.next = next
    def Flat():
        model.add(Flatten())
    Flat()

class DenseNode(): # this can also be k long
    def __init__(self,val = 0, next = None):
        self.val = val
        self.next = next
    def output():
        model.add(Dense(12))
        model.add(Activation("relu"))
    output()
    
class OutputNode(): # we need only one output node
    def __init__(self,val = 0 , next = None):
        self.val = val
        self.next = next
    def outputnode():
        model.add(Dense(1))
        model.add(Activation("sigmoid"))
    outputnode()
# the linked list will at least be 5 nodes long
def create_Model(hidden_value , dense_value):
    count = 0 # to help us control the node numbers
    cur = ConvNode(count) # conv
    count += 1
    dummy = cur

    for conv in range(hidden_value): # creating amount of 4 X hidden_value layers
        cur.next = HiddenNode(count)
        count += 1
        cur = cur.next
    cur.next = FlattenNode(count)  # flatten layer
    count += 1
    cur = cur.next
    for dense in range(dense_value): # dense layers
        cur.next = DenseNode(count)
        count += 1
        cur = cur.next
    cur.next = OutputNode(count) # output layer
    count += 1
    cur = cur.next
    return pretty(dummy)
create_Model(5,5) # controling hidden and dense layers

# so now we have created the model within the linked list

model.compile(loss = "binarycrossentropy" , optimizer = "adam" , metrics = ["accuracy"])

model.fit(



    
        
    
    
    
        


        
    
        
        
        




0->1->2->3->4->5->6->7->8->9->10->11->12->None
